In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.dex_proxy_contract import DexProxyContract
from contracts.locked_asset_contract import LockedAssetContract
import json
from utils.utils_chain import string_to_hex, WrapperAddress, decode_merged_attributes
from utils.decoding_structures import ENERGY_ENTRY
from concurrent.futures import ThreadPoolExecutor

context = Context()

In [ ]:
from pprint import pprint
from typing import List
from multiversx_sdk import ProxyNetworkProvider
from multiversx_sdk.network_providers.tokens import NonFungibleTokenOfAccountOnNetwork

energy_contract: SimpleLockEnergyContract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
locked_token_factory_contract: LockedAssetContract
locked_token_factory_contract = context.get_contracts(config.LOCKED_ASSETS)[0]
proxy_contract: DexProxyContract
proxy_contract = context.get_contracts(config.PROXIES_V2)[0]
proxy_old_contract: DexProxyContract
proxy_old_contract = context.get_contracts(config.PROXIES)[0]
pprint(energy_contract.get_config_dict())
pprint(locked_token_factory_contract.get_config_dict())
pprint(proxy_contract.get_config_dict())
pprint(proxy_old_contract.get_config_dict())

class processed_user:
    tokens: List[NonFungibleTokenOfAccountOnNetwork]
    energy: int
    address: str
    total_locked_tokens: int
    metabonding_tokens: int
    
    def __init__(self, address: str) -> None:
        self.address = address
        self.tokens = []
        self.energy = 0
        self.total_locked_tokens = 0
        self.metabonding_tokens = 0

    def set_tokens(self, tokens: List[NonFungibleTokenOfAccountOnNetwork]):
        self.tokens = tokens

    def set_energy(self, energy: int, total_locked_tokens: int):
        self.energy = energy
        self.total_locked_tokens = total_locked_tokens

    def set_metabonding_tokens(self, metabonding_tokens: int):
        self.metabonding_tokens = metabonding_tokens

    def retrieve_tokens(self, proxy: ProxyNetworkProvider):
        self.tokens = proxy.get_nonfungible_tokens_of_account(self.address)

Get energy factory state

In [ ]:
# get state of energy contract
from tools.common import fetch_contract_state

fetch_contract_state(energy_contract.address, "energy-contract-state", context.network_provider)

Retrieve searched user tokens

In [ ]:
searched_tokens = []
searched_tokens.extend(proxy_contract.locked_tokens)
searched_tokens.extend([proxy_contract.proxy_lp_token, proxy_contract.proxy_farm_token])
searched_tokens.extend([proxy_old_contract.proxy_lp_token, proxy_old_contract.proxy_farm_token])
searched_tokens.extend(["EGLDMEXFL-ef2065", "EGLDUSDCFL-f0031c", "MEXFARML-28d646", "EGLDRIDEFL-74b819", "ITHWEGLDFL-332f38", "UTKWEGLDFL-082aec", "CRTWEGLDFL-e0454e", "ASHWEGLDFL-cf0194", "LKFARM-321c30"])

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "accounts_with_diffs_fixed.json", "r") as f:
    accounts_with_diffs = json.load(f)

processed_users = []

def retrieve_user_tokens(address: str):
    try:
        user = processed_user(address)
        tokens = context.network_provider.proxy.get_nonfungible_tokens_of_account(WrapperAddress(address))
        user.set_tokens([token for token in tokens if token.collection in searched_tokens])
    except Exception as e:
        print(f"Error retrieving tokens for {user.address}: {str(e)}")
    return user

with ThreadPoolExecutor(max_workers=100) as executor:
    total_users = len(accounts_with_diffs.keys())
    for i, user in enumerate(executor.map(retrieve_user_tokens, accounts_with_diffs.keys())):
        print(f"Processing user {i+1}/{total_users}", end="\r")
        processed_users.append(user)

Decode user energy

In [34]:
config.DEFAULT_WORKSPACE
with open(config.DEFAULT_WORKSPACE / "deploy" / "configs-mainnet" / "upgrader_outputs" / "energy-contract-state.json", "r") as f:
    energy_contract_state = json.load(f)
    
for user in processed_users:
    user_address = WrapperAddress(user.address)
    user_energy_key = f"{string_to_hex("userEnergy")}{user_address.to_hex()}"
    user_energy_raw = energy_contract_state.get(user_energy_key)

    user_energy_entry = decode_merged_attributes(user_energy_raw, ENERGY_ENTRY)
    user.set_energy(user_energy_entry.get("amount"), user_energy_entry.get("total_locked_tokens"))

Retrieve metabonding

In [ ]:
from multiversx_sdk import ContractQueryBuilder
from utils.utils_chain import base64_to_hex

def retrieve_metabonding_tokens(user: processed_user):
    try:
        query = ContractQueryBuilder(
            WrapperAddress("erd1qqqqqqqqqqqqqpgqt7tyyswqvplpcqnhwe20xqrj7q7ap27d2jps7zczse"),   # metabonding contract
            "getStakedAmountForUser",
            [WrapperAddress(user.address)]
        ).build()
        result = context.network_provider.proxy.query_contract(query)
        amount_hex = base64_to_hex(result.return_data[0])
        if amount_hex == "":
            amount_hex = "0"
        user.set_metabonding_tokens(int(amount_hex, 16))
    except Exception as e:
        print(f"Error retrieving metabonding tokens for {user.address}: {str(e)}")
    return user

new_processed_users = []
with ThreadPoolExecutor(max_workers=100) as executor:
    total_users = len(processed_users)
    for i, user in enumerate(executor.map(retrieve_metabonding_tokens, processed_users)):
        print(f"Processing user {i+1}/{total_users}", end="\r")
        new_processed_users.append(user)

processed_users = new_processed_users

Process statistics

In [ ]:
users_with_zero_energy_no_tokens = []
users_with_zero_energy_with_tokens = []
users_with_energy_no_tokens = []
users_with_energy_with_tokens = []

for user in processed_users:
    user_dict = {
        "address": user.address,
        "energy": user.energy,
        "total_locked_tokens": user.total_locked_tokens,
        "tokens": [token.identifier for token in user.tokens],
        "metabonding_tokens": user.metabonding_tokens
    }
    if user.energy == 0:
        if len(user.tokens) > 0:
            users_with_zero_energy_with_tokens.append(user_dict)
        else:
            users_with_zero_energy_no_tokens.append(user_dict)
    else:
        if len(user.tokens) > 0:
            users_with_energy_with_tokens.append(user_dict)
        else:
            users_with_energy_no_tokens.append(user_dict)

print(f"Users with zero energy and no tokens: {len(users_with_zero_energy_no_tokens)}")
print(f"Users with zero energy and with tokens: {len(users_with_zero_energy_with_tokens)}")
print(f"Users with energy and no tokens: {len(users_with_energy_no_tokens)}")
print(f"Users with energy and with tokens: {len(users_with_energy_with_tokens)}")

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "users_with_zero_energy_no_tokens.json", "w") as f:
    json.dump(users_with_zero_energy_no_tokens, f, indent=4)

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "users_with_zero_energy_with_tokens.json", "w") as f:
    json.dump(users_with_zero_energy_with_tokens, f, indent=4)

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "users_with_energy_no_tokens.json", "w") as f:
    json.dump(users_with_energy_no_tokens, f, indent=4)

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "users_with_energy_with_tokens.json", "w") as f:
    json.dump(users_with_energy_with_tokens, f, indent=4)

In [ ]:
energy_change = "-1380949284391887276302652"
print(int(energy_change))
print(string_to_hex(energy_change))
energy_change = int(string_to_hex(energy_change), 16) * -1
print(energy_change)
print(int("2d31333830393439323834333931383837323736333032363532", 16))

upgrade metabonding contract

In [ ]:
from utils.utils_tx import upgrade_call
from multiversx_sdk import CodeMetadata

metabonding_address = "erd1qqqqqqqqqqqqqpgqt7tyyswqvplpcqnhwe20xqrj7q7ap27d2jps7zczse"
altered_bytecode = config.HOME / "temp/mx-exchange-sc/farm-staking/metabonding-staking/output/metabonding-staking.wasm"
metadata = CodeMetadata(upgradeable=True, readable=True, payable=False, payable_by_contract=True)

upgrade_call("", context.network_provider.proxy, 100000000, context.deployer_account, WrapperAddress(metabonding_address), altered_bytecode, metadata, [])

exit metabonding

In [ ]:
from utils.utils_chain import Account
from utils.utils_tx import endpoint_call

metabonding_address = "erd1qqqqqqqqqqqqqpgqt7tyyswqvplpcqnhwe20xqrj7q7ap27d2jps7zczse"
for user in users_with_energy_no_tokens:
    if user.get("total_locked_tokens") < 10000:
        continue
    temp_account = Account(pem_file=config.DEFAULT_ADMIN)
    temp_account.address = WrapperAddress(user.get("address"))
    temp_account.sync_nonce(context.network_provider.proxy)

    endpoint_call(context.network_provider.proxy, 20000000, temp_account, WrapperAddress(metabonding_address), "unstake", [])
    endpoint_call(context.network_provider.proxy, 20000000, temp_account, WrapperAddress(metabonding_address), "unbond", [])

Get metabonding state

In [ ]:
from tools.common import fetch_contract_state

fetch_contract_state("erd1qqqqqqqqqqqqqpgqt7tyyswqvplpcqnhwe20xqrj7q7ap27d2jps7zczse", "metabonding-state", context.network_provider)

In [ ]:
from utils.utils_chain import Account
from utils.utils_tx import endpoint_call
from tools.runners.common_runner import fund_shadowfork_accounts, ExportedAccount

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "all_users_energy.json", "r") as f:
    loaded_users = json.load(f)
users = [user["address"] for user in loaded_users if user["total_locked_tokens"] != 0]

fund_shadowfork_accounts([ExportedAccount(user, 0, 0, []) for user in users])
input("Funded accounts, press Enter to continue...")

metabonding_address = "erd1qqqqqqqqqqqqqpgqt7tyyswqvplpcqnhwe20xqrj7q7ap27d2jps7zczse"
for user in users:
    temp_account = Account(pem_file=config.DEFAULT_ADMIN)
    temp_account.address = WrapperAddress(user)
    temp_account.sync_nonce(context.network_provider.proxy)

    endpoint_call(context.network_provider.proxy, 20000000, temp_account, WrapperAddress(metabonding_address), "unstake", [])
    endpoint_call(context.network_provider.proxy, 20000000, temp_account, WrapperAddress(metabonding_address), "unbond", [])

exit lk farm egldmexf

In [ ]:
from utils.utils_chain import Account, WrapperAddress, get_all_token_nonces_details_for_account
from utils.utils_tx import endpoint_call
from contracts.dex_proxy_contract import DexProxyContract, DexProxyExitFarmEvent
from contracts.farm_contract import FarmContract, FarmContractVersion

temp_account = Account(pem_file=config.DEFAULT_ADMIN)
temp_account.address = WrapperAddress("erd1sv9mt3na7d754wx7kk9suvj4azf4hsz5xxvx038v5z4jgxy0uelqjd7q4w")
temp_account.sync_nonce(context.network_provider.proxy)

tokens = get_all_token_nonces_details_for_account("LKFARM-9d1ea8", temp_account.address.bech32(),context.network_provider.proxy)
farm_contract = FarmContract("EGLDMEX-0be9e5", "EGLDMEXF-a4d81e", "MEX-455c57", "erd1qqqqqqqqqqqqqpgqnqvjnn4haygsw2hls2k9zjjadnjf9w7g2jpsmc60a4", FarmContractVersion.V2Boosted)

proxy_contract: DexProxyContract = context.get_contracts(config.PROXIES)[0]
event = DexProxyExitFarmEvent(farm_contract, tokens[0].get("tokenIdentifier"), tokens[0].get("nonce"), int(tokens[0].get("balance")))
proxy_contract.exit_farm_proxy(temp_account, context.network_provider.proxy, event)

exit lklp

In [ ]:
from contracts.pair_contract import PairContract
from contracts.dex_proxy_contract import DexProxyRemoveLiquidityEvent

pair_contract: PairContract = context.get_contracts(config.PAIRS_V2)[0]

tokens = get_all_token_nonces_details_for_account("LKLP-03a2fa", temp_account.address.bech32(), context.network_provider.proxy)

event = DexProxyRemoveLiquidityEvent(pair_contract, int(tokens[0].get("balance")), tokens[0].get("nonce"), 1, 1)
proxy_contract.remove_liquidity_proxy(temp_account, context.network_provider.proxy, event)

Export all users energy from energy contract

In [ ]:
config.DEFAULT_WORKSPACE
with open(config.DEFAULT_WORKSPACE / "deploy" / "configs-mainnet" / "upgrader_outputs" / "energy-contract-state.json", "r") as f:
    energy_contract_state = json.load(f)

users = []
for key, value in energy_contract_state.items():
    if string_to_hex("userEnergy") not in key:
        continue
    user_address = WrapperAddress.from_hex(key.replace(string_to_hex("userEnergy"), ""))
    user_energy_entry = decode_merged_attributes(value, ENERGY_ENTRY)

    user_dict = {
        "address": user_address.bech32(),
        "energy": user_energy_entry.get("amount"),
        "last_update_epoch": user_energy_entry.get("last_update_epoch"),
        "total_locked_tokens": user_energy_entry.get("total_locked_tokens"),
        "tokens": [],
        "metabonding_tokens": 0
    }
    users.append(user_dict)

with open(config.DEFAULT_WORKSPACE / "energy-fix" / "all_users_energy.json", "w") as f:
    json.dump(users, f, indent=4)

print(f"Done. Found {len(users)} users")


In [ ]:
with open(config.DEFAULT_WORKSPACE / "energy-fix" / "all_users_energy.json", "r") as f:
    users = json.load(f)

i = 0
for user in users:
    total_locked_tokens = user.get("total_locked_tokens")
    energy = user.get("energy")
    if energy > 10000000:
        i += 1
        print(user)

print(f"Selected users: {i}")

In [ ]:
decode_merged_attributes("0000000c0291dbdb7a8b0eeb03f42128000000000000035e0000000b0c3a54f8c2d298d44be9c6", ENERGY_ENTRY)